In [27]:
#MonteCarlo Time of Arrival GPS Sim for Receiver Localization. 
#Estimating out recievier position, clock offset, and TEC values

#Fausto Vega

using SatelliteDynamics
using LinearAlgebra
using PlotlyJS
using DelimitedFiles
using Distributions

In [28]:
# Declare simulation initial Epoch. Start time
epc0 = Epoch(2019, 1, 1, 12, 0, 0, 0.0)  #year, month, day, hour, minute, seconds, nanoseconds 

Epoch(2019-01-01T11:59:23.000Z)

In [29]:
# Declare initial state in terms of osculating orbital elements
#semi-major axis, eccentricity, inclination, RAAN, Argument of perigee, Mean anomaly

#1 and 2 on the same orbit, different initial mean anomaly

#previous orbits working. Not 100 km apart at the equator (hard to do in practice)
#iss1 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 185]; #converges at 189.0859
#iss2 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 205];
#iss3 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 190]; # converges a t 200.0859
#iss4 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 215]; # converges a t 200.0859

#approximately 500 km elevation from surface of Earth and around 100 km seperation at equator
#working good
iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 190]; 
iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 200];
iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 193]; 
iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 203]; 

E1 = anomaly_mean_to_eccentric(iss1[6], iss1[2], use_degrees=true);
E2 = anomaly_mean_to_eccentric(iss2[6], iss2[2], use_degrees=true);
E3 = anomaly_mean_to_eccentric(iss3[6], iss3[2], use_degrees=true);
E4 = anomaly_mean_to_eccentric(iss4[6], iss4[2], use_degrees=true);

In [30]:
#15 degree seperation (~1800 km)
#iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 187.5]; 
#iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 202.5];
#iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 190.5]; 
#iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 205.5]; 

#10 degree seperation (~1200 km)
#iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 190]; 
#iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 200];
#iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 193]; 
#iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 203]; 

#5 degree seperation (~600 km)
#iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 192.5]; 
#iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 197.5];
#iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 195.5]; 
#iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 200.5]; 

#2.5 degree seperation (~300 km)

#iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 193.75]; 
#iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 196.25];
#iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 196.75]; 
#iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 199.25]; 

#1.25 degree seperation (~150 km)

#iss1 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 194.375]; 
#iss2 = [6871e3, 0.0004879, 90.6391, 194.5859, 151.2014, 195.625];
#iss3 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 197.375]; 
#iss4 = [6871e3, 0.0004879, 90.6391, 195.5859, 151.2014, 198.625]; 

In [31]:
# Convert osculating elements to Cartesean state
# returns position and velocity (m, m/s)

eci0_1 = sOSCtoCART(iss1, use_degrees=true)
eci0_2 = sOSCtoCART(iss2, use_degrees=true)
eci0_3 = sOSCtoCART(iss3, use_degrees=true)
eci0_4 = sOSCtoCART(iss4, use_degrees=true)

6-element Vector{Float64}:
      -6.585092459477662e6
      -1.8449166087197426e6
 -697065.2121487064
    -764.9760976521648
    -125.67504089818506
    7573.5676513547705

In [32]:
#compute the satellite orbital period given the semi-major axis

#same for all 4 satellites since they have the same semi-major axis
T = orbit_period(iss1[1])

#final time for one orbit. Adds initial time to the orbital period

epcf = epc0 + T

Epoch(2019-01-01T13:33:51.144Z)

In [33]:
# Create an EarthInertialState orbit propagagator
#needs initial epoch of state and the state vector
orb1  = EarthInertialState(epc0, eci0_1, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb2  = EarthInertialState(epc0, eci0_2, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb3  = EarthInertialState(epc0, eci0_3, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb4  = EarthInertialState(epc0, eci0_4, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

# Propagate the orbit
# orbit until the final time

t_1, epc_1, eci_1 = sim!(orb1, epcf);
t_2, epc_2, eci_2 = sim!(orb2, epcf);
t_3, epc_3, eci_3 = sim!(orb3, epcf);
t_4, epc_4, eci_4 = sim!(orb4, epcf);

In [34]:
#equator position in cartesian coordinates
#tag =[-6.128804e6, -1.590206e6, 776.1502e3]

#Equitorial position
tag_geof = [-165.4545, 0, 0]

#North pole position
#tag_geof = [-165.4545, 46.98849, 0]

tag = sGEOCtoECEF(tag_geof, use_degrees = true)

3-element Vector{Float64}:
 -6.173708155673107e6
 -1.6018611673126891e6
  0.0

In [35]:
#Equitorial Position
#In m
x0= tag[1] 
y0 = tag[2]
z0 = tag[3]
t0 = 1e-5

r0 = [x0, y0, z0, t0]

centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

#Guess
xyz = sGEOCtoECEF(geodetic, use_degrees = true)

println(xyz)

[-6.012592823500298e6, -1.636770880711428e6, -1.360272110137101e6]


In [36]:
#Plot Satellite 1 and 2 Orbit
sat1 = scatter(x=eci_1[1,:], y=eci_1[2,:], z=eci_1[3,:], type="scatter3d", mode="lines", name="orbit 1&2")

#Plot Satellite 3 and 4 Orbit
sat3 = scatter(x=eci_3[1,:], y=eci_3[2,:], z=eci_3[3,:], type="scatter3d", mode="lines", name="orbit 3&4")

#Plot all 4 satellite initial positions
satellites = scatter(x=[eci_1[1,1],eci_2[1,1], eci_3[1,1], eci_4[1,1]], y=[eci_1[2,1],eci_2[2,1], eci_3[2,1], eci_4[2,1]],z=[eci_1[3,1],eci_2[3,1], eci_3[3,1], eci_4[3,1]], mode="markers", marker_size = 4, type="scatter3d", name="satellites")

#Plot Tag position
tag = scatter(x = [r0[1], r0[1]], y = [r0[2], r0[2]], z = [r0[3], r0[3]], type="scatter3d", name="tag", mode="markers", marker_size=5)

#Plot the Guess
guess = scatter(x = [xyz[1], xyz[1]], y = [xyz[2], xyz[2]], z = [xyz[3], xyz[3]], type="scatter3d", name="guess", mode="markers", marker_size=5)

#Plotting the sphere (Earth)
n = 100
u = range(-π, π; length = n)
v = range(0, π; length = n)
x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)' 
z = ones(n) * cos.(v)' 

earth = surface(z=z*6371000, x=x*6371000, y=y*6371000, showscale = false)

plot([sat1,sat3, satellites, tag, earth, guess])

data: [
  "scatter3d with fields mode, name, type, x, y, and z",
  "scatter3d with fields mode, name, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z",
  "surface with fields showscale, type, x, y, and z",
  "scatter3d with fields marker, mode, name, type, x, y, and z"
]

layout: "layout with fields margin and template"

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

...

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

In [37]:
pose1 = eci_1[:,1][1:3]

pose2 = eci_2[:,1][1:3]
pose4 = eci_4[:,1][1:3]

distance = norm(pose2-pose4) 

a_distance = norm(pose1 - pose2)

1.1971320321041318e6

In [38]:
### Finding the Elevation Angle

function elevation_angle(satposes, r0)
    
    elevangle = zeros(4)
    #x = 0
    #y = 0

    #A,B = horizon(r0[1:3])
    #z =  A*(x-r0[1]) + B*(y-r0[2]) + r0[3]

    #vector in the horizon plane
    #vec_inplane = [x,y,z] - r0[1:3]

    #vector between tag and the satellite
    normalvec = r0
    
    for i in 1:4
        
        vector = satposes[i,:] - r0[1:3]
        
        #Find the angle between the normal and the vector going from the tag to the satellte
        theta = acos(dot(normalvec,vector)/(norm(normalvec)*norm(vector))) * (180/pi)
        
        #save elevation angle
        elevangle[i] = theta
    end
    
    return elevangle
end

elevation_angle (generic function with 1 method)

In [39]:
#Testing elevation angle function
satposes = [eci_1[1,1] eci_1[2,1] eci_1[3,1];eci_2[1,1] eci_2[2,1] eci_2[3,1];eci_3[1,1] eci_3[2,1] eci_3[3,1];eci_4[1,1] eci_4[2,1] eci_4[3,1]]

satposes2 = [eci_1[1,2] eci_1[2,2] eci_1[3,2];eci_2[1,2] eci_2[2,2] eci_2[3,2];eci_3[1,2] eci_3[2,2] eci_3[3,2];eci_4[1,2] eci_4[2,2] eci_4[3,2]]

satposes3 = [eci_1[1,3] eci_1[2,3] eci_1[3,3];eci_2[1,3] eci_2[2,3] eci_2[3,3];eci_3[1,3] eci_3[2,3] eci_3[3,3];eci_4[1,3] eci_4[2,3] eci_4[3,3]]

elev_angles = elevation_angle(satposes, r0[1:3])

elev_angles2= elevation_angle(satposes2, r0[1:3])

elev_angles3= elevation_angle(satposes3, r0[1:3])

println(elev_angles)

println(elev_angles2)

println("dot: ", elev_angles - elev_angles2)
println("dot: ", elev_angles2 - elev_angles3)

#check all angles to see if it is on the horizon 
inhorizon = all(x->x<80, elev_angles)

[86.66698734179924, 68.51644660741826, 82.88110109625767, 57.083922389433035]
[86.59281807323111, 68.32121894190128, 82.79115822397472, 56.763406931949184]
dot: [0.07416926856812722, 0.19522766551698112, 0.08994287228294695, 0.32051545748385024]
dot: [0.07444050539250213, 0.19714524930199673, 0.09037504749994696, 0.3244394712776213]


false

In [40]:
#Obtain the orbital elements after each time step 
dim = size(t_1)[1]

all_elements = zeros(6,dim)
for i in 1:dim
    
    orbital_elements = sCARTtoOSC(eci_1[:,1], use_degrees=true)
    all_elements[:,i] = orbital_elements
end 

In [41]:
#scaled
mu = 8e-6
sigma = 3e-6
lb = 3e-6
ub = 13e-6
d = Truncated(Normal(mu,sigma), lb, ub)

hi = 350e3 #in meters
#Ip = zeros(8)
#OF = zeros(8)
#Iz = zeros(8)
#Ip_scaled = zeros(8)

350000.0

In [42]:
#plot delta d over speed of light
# d versus f plot
#10 cm delta d
using ForwardDiff
#using Plots
using BlockDiagonals

#time_scale = 1/47.004
#distance_scale = 6378.14 

distance_scale = R_EARTH*(1e-3) # scales km to custom scale
time_scale = 1/(C_LIGHT/R_EARTH) #scales seconds to custom scale

c = 1 # 1 distance_unit/ 1 time_unit

1

In [43]:
dunit = 1/distance_scale
tunit = 1/time_scale
speed = dunit/tunit

3.33564095198152e-6

In [44]:
function measurment_residual(r0, satposes,zenith, t)
    
    omega = OMEGA_EARTH*time_scale
    
    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end
    
    A1 = [cos(omega*t[1]) sin(omega*t[1]) 0;
          -sin(omega*t[1]) cos(omega*t[1]) 0;
          0 0 1];
    A2 = [cos(omega*t[2]) sin(omega*t[2]) 0;
          -sin(omega*t[2]) cos(omega*t[2]) 0;
          0 0 1];
    A3 = [cos(omega*t[3]) sin(omega*t[3]) 0;
          -sin(omega*t[3]) cos(omega*t[3]) 0;
          0 0 1];
    A4 = [cos(omega*t[4]) sin(omega*t[4]) 0;
          -sin(omega*t[4]) cos(omega*t[4]) 0;
          0 0 1];
    A5 = [cos(omega*t[5]) sin(omega*t[5]) 0;
          -sin(omega*t[5]) cos(omega*t[5]) 0;
          0 0 1];
    A6 = [cos(omega*t[6]) sin(omega*t[6]) 0;
          -sin(omega*t[6]) cos(omega*t[6]) 0;
          0 0 1];
    A7 = [cos(omega*t[7]) sin(omega*t[7]) 0;
          -sin(omega*t[7]) cos(omega*t[7]) 0;
          0 0 1];
    A8 = [cos(omega*t[8]) sin(omega*t[8]) 0;
          -sin(omega*t[8]) cos(omega*t[8]) 0;
          0 0 1];
    
    res1 = norm(r0[1:3] - A1*satposes[1,1:3]) - c*(t[1] - r0[4]) + (scale_f1*(r0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(r0[1:3] - A2*satposes[2,1:3]) - c*(t[2] - r0[4]) + (scale_f1*(r0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(r0[1:3] - A3*satposes[3,1:3]) - c*(t[3] - r0[4]) + (scale_f1*(r0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(r0[1:3] - A4*satposes[4,1:3]) - c*(t[4] - r0[4]) + (scale_f1*(r0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(r0[1:3] - A5*satposes[1,1:3]) - c*(t[5] - r0[4]) + (scale_f2*(r0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(r0[1:3] - A6*satposes[2,1:3]) - c*(t[6] - r0[4]) + (scale_f2*(r0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(r0[1:3] - A7*satposes[3,1:3]) - c*(t[7] - r0[4]) + (scale_f2*(r0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(r0[1:3] - A8*satposes[4,1:3]) - c*(t[8] - r0[4]) + (scale_f2*(r0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

measurment_residual (generic function with 1 method)

In [45]:
function residual(x, sat_pose_t, zenith)
    
    omega = OMEGA_EARTH*time_scale
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
          -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
          -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
          -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
          -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
          0 0 1];
    
    res1 = norm(x[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - x[4]) + (scale_f1*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(x[1:3] - A2*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - x[4]) + (scale_f1*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(x[1:3] - A3*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - x[4]) + (scale_f1*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(x[1:3] - A4*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - x[4]) + (scale_f1*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(x[1:3] - A5*sat_pose_t[1,1:3]) - c*(sat_pose_t[5,4] - x[4]) + (scale_f2*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(x[1:3] - A6*sat_pose_t[2,1:3]) - c*(sat_pose_t[6,4] - x[4]) + (scale_f2*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(x[1:3] - A7*sat_pose_t[3,1:3]) - c*(sat_pose_t[7,4] - x[4]) + (scale_f2*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(x[1:3] - A8*sat_pose_t[4,1:3]) - c*(sat_pose_t[8,4] - x[4]) + (scale_f2*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

residual (generic function with 1 method)

In [46]:
function residual_ms(x, sat_pose_t, zenith)
    
    #Everything in meters and seconds
    
    c = 299792458 #speed of light in m/s
    
    omega = OMEGA_EARTH
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
          -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
          -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
          -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
          -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
          0 0 1];
    
    res1 = norm(x[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - x[4]) + (scale_f1*(x[5]/cosd(angles[1])));
    res2 = norm(x[1:3] - A2*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - x[4]) + (scale_f1*(x[6]/cosd(angles[2])));
    res3 = norm(x[1:3] - A3*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - x[4]) + (scale_f1*(x[7]/cosd(angles[3])));
    res4 = norm(x[1:3] - A4*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - x[4]) + (scale_f1*(x[8]/cosd(angles[4])));
    res5 = norm(x[1:3] - A5*sat_pose_t[1,1:3]) - c*(sat_pose_t[5,4] - x[4]) + (scale_f2*(x[5]/cosd(angles[1])));
    res6 = norm(x[1:3] - A6*sat_pose_t[2,1:3]) - c*(sat_pose_t[6,4] - x[4]) + (scale_f2*(x[6]/cosd(angles[2])));
    res7 = norm(x[1:3] - A7*sat_pose_t[3,1:3]) - c*(sat_pose_t[7,4] - x[4]) + (scale_f2*(x[7]/cosd(angles[3])));
    res8 = norm(x[1:3] - A8*sat_pose_t[4,1:3]) - c*(sat_pose_t[8,4] - x[4]) + (scale_f2*(x[8]/cosd(angles[4])));
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

residual_ms (generic function with 1 method)

In [47]:
function get_time(r0_TEC, satposes, zenith)
    
    n = 1000

    time = [zeros(8) for i = 1:1000]
    Rt = [zeros(8) for i = 1:1000]

    time[1] = [0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale] #initial guess

    Rt[1] = measurment_residual(r0_TEC, satposes, zenith, time[1])

    iters = 0

    for i=1:n

        Rt[i] = measurment_residual(r0_TEC, satposes, zenith, time[i])

        #println("this is norm of residual: ", norm(Rt[i]))
        
        iters += 1

        if(norm(Rt[i]) < 1e-6)
            break
        end

        jacobian = ForwardDiff.jacobian(dt -> measurment_residual(r0_TEC, satposes, zenith, dt), time[i])

        deltat = (jacobian)\-Rt[i]

        time[i+1]  = time[i] + deltat
    end
    
    Rt = Rt[1:iters]
    time = time[1:iters]
    time_measurment = time[end]
    
    return time_measurment
    
end

get_time (generic function with 1 method)

In [48]:
#Equitorial Position

#In km
x0= x0*1e-3
y0 = y0*1e-3
z0 = z0*1e-3
t0 = 1e-5

#r0 = [x0,y0,z0,t0]
r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, t0/time_scale]

4-element Vector{Float64}:
 -0.9679486083847263
 -0.2511487826487322
  0.0
  0.0004700314384940316

In [49]:
#frequency variables
f1 = 400e6 #in Hz
f2 = 600e6  #in Hz

6.0e8

In [50]:
#clock error distribution
mu_t = 0
sigma_t = 20e-9 #RMSE for Novatel GPS receiver
lb_t = 1e-9
ub_t = 30e-9
#d_t = Truncated(Normal(mu_t,sigma_t), lb_t, ub_t)
d_t = Normal(mu_t, sigma_t)

Normal{Float64}(μ=0.0, σ=2.0e-8)

In [51]:
# n - 1000
# sat1poses = zeros(4,n)
# sat2poses = zeros(4,n)
# sat3poses = zeros(4,n)
# sat4poses = zeros(4,n)
# sat5poses = zeros(4,n)
# sat6poses = zeros(4,n)
# sat7poses = zeros(4,n)
# sat8poses = zeros(4,n)

In [52]:
function receiver_pose(all_sats_scaled, guess, elev_angles, TEC) # remember to scale the guess to custom units

    n = 1000 # number of iterations
    #all_r0 = NaN*[zeros(4) for i = 1:n]
    all_r0 = [zeros(8) for i = 1:n]

    all_sats_noise = zeros(8,4)
    
    sat1poses = zeros(4,n)
    sat2poses = zeros(4,n)
    sat3poses = zeros(4,n)
    sat4poses = zeros(4,n)
    sat5poses = zeros(4,n)
    sat6poses = zeros(4,n)
    sat7poses = zeros(4,n)
    sat8poses = zeros(4,n)


    iters = 0

     #Monte Carlo Simulation
    for i in 1:n
        b = 0.01
        c=0.5
        β = 10
        α = 1
        
        #create noise from random distribution
        #scaled to the variable (0.1m error for distance & 1e-11 for time)
        gpsnoise = randn(12)*(1e-4/distance_scale) #adding a 0.1 meter (10 cm) of noise 
        #clocknoise = randn(8)*(1e-11/time_scale) #original working (may not be accurate)
        #clocknoise = randn(8)*1e-9/time_scale # from Novatel resource ~20 ns accuracy
        clocknoise = rand(d_t,8)/time_scale
        
        sat1_noise = [all_sats_scaled[1,1]+gpsnoise[1],all_sats_scaled[1,2] + gpsnoise[2],all_sats_scaled[1,3]+gpsnoise[3], all_sats_scaled[1,4] + clocknoise[1]]
        sat2_noise =[all_sats_scaled[2,1]+gpsnoise[4],all_sats_scaled[2,2] + gpsnoise[5],all_sats_scaled[2,3]+gpsnoise[6], all_sats_scaled[2,4] + clocknoise[2]]
        sat3_noise = [all_sats_scaled[3,1]+gpsnoise[7],all_sats_scaled[3,2] + gpsnoise[8],all_sats_scaled[3,3]+gpsnoise[9], all_sats_scaled[3,4] + clocknoise[3]]
        sat4_noise = [all_sats_scaled[4,1]+gpsnoise[10],all_sats_scaled[4,2] + gpsnoise[11],all_sats_scaled[4,3]+gpsnoise[12], all_sats_scaled[4,4] + clocknoise[4]]

        sat5_noise = [all_sats_scaled[1,1]+gpsnoise[1],all_sats_scaled[1,2] + gpsnoise[2],all_sats_scaled[1,3]+gpsnoise[3], all_sats_scaled[5,4] + clocknoise[5]]
        sat6_noise =[all_sats_scaled[2,1]+gpsnoise[4],all_sats_scaled[2,2] + gpsnoise[5],all_sats_scaled[2,3]+gpsnoise[6], all_sats_scaled[6,4] + clocknoise[6]]
        sat7_noise = [all_sats_scaled[3,1]+gpsnoise[7],all_sats_scaled[3,2] + gpsnoise[8],all_sats_scaled[3,3]+gpsnoise[9], all_sats_scaled[7,4] + clocknoise[7]]
        sat8_noise = [all_sats_scaled[4,1]+gpsnoise[10],all_sats_scaled[4,2] + gpsnoise[11],all_sats_scaled[4,3]+gpsnoise[12], all_sats_scaled[8,4] + clocknoise[8]]

        all_sats_noise[:,:] = vcat(sat1_noise',sat2_noise',sat3_noise',sat4_noise',sat5_noise',sat6_noise',sat7_noise',sat8_noise')

        sat1poses[:,i] = sat1_noise
        sat2poses[:,i] = sat2_noise
        sat3poses[:,i] = sat3_noise
        sat4poses[:,i] = sat4_noise
        
        sat5poses[:,i] = sat5_noise
        sat6poses[:,i] = sat6_noise
        sat7poses[:,i] = sat7_noise
        sat8poses[:,i] = sat8_noise
        
        #second frequency same 4 satellite config
        #sat5poses[:,i] = sat5_noise
        #sat6poses[:,i] = sat6_noise
        #sat7poses[:,i] = sat7_noise
        #sat8poses[:,i] = sat8_noise
        
#         #second frequency same 4 satellite config
#         sat5poses[:,i] = sat5_noise
#         sat6poses[:,i] = sat6_noise
#         sat7poses[:,i] = sat7_noise
#         sat8poses[:,i] = sat8_noise

        #X = NaN*[zeros(4) for i = 1:100]
        #R = NaN*[zeros(4) for i = 1:100]
        
        X = NaN*[zeros(8) for i = 1:1000]
        R = NaN*[zeros(8) for i = 1:1000]
        
        #Centroid guess
        X[1] = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 0.0001/time_scale, 3e-6, 3.5e-6, 2e-6, 4e-6]

        #Initial residual
        
        R[1] = residual(X[1], all_sats_noise, elev_angles)

        iters = 0

        for k=1:1000
            
            R[k] = residual(X[k], all_sats_noise, elev_angles)
            
            #println("this is residual: ", norm(R[k]))

            iters += 1
        
            #original tolerancec 1e-12
            
            if(norm(R[k]) < 1e-10)

                break

            end

            jacobian = ForwardDiff.jacobian(dx -> residual(dx, all_sats_noise, elev_angles), X[k])

            conditionnum = cond(jacobian)
            
            #println("this is the jacobian: ", jacobian)
            
            #println("condition number: ", conditionnum)
            
            #println("This is R[k]: ", R[k])
            
            #println("this is TEC: ", TEC)
            #println("this is X[k]: ", X[k])
            
            #println("this is R[k]: ", norm(R[k]))
            
            #println("this is jacobian: ", jacobian)
            
            deltax = (jacobian)\-R[k]
            
            #println("this is deltax: ", deltax)
            
            while norm(residual(X[k] + α*deltax, all_sats_noise, elev_angles)) > norm(residual(X[k], all_sats_noise, elev_angles) + b*α*jacobian'*deltax)

                α = c*α
                #print("this is alpha: ", α)
            end

            #println("this is delta x: ", deltax)
            X[k+1] = X[k] + α*deltax
            
            #println("this is delta x+1: ", X[k+1])
            
        end  
        
        R = R[1:iters]
        X = X[1:iters]

        all_r0[i] = X[end]

    end
    x_values = zeros(n)
    y_values = zeros(n)
    z_values = zeros(n)

    for j in 1:n

        x_values[j] = all_r0[j][1]
        y_values[j] = all_r0[j][2]
        z_values[j] = all_r0[j][3]

    end
    
    #mean calculation
    mean = sum(all_r0, dims = 1)/ n
    mean = mean[1]
    mean_rescaled = [mean[1:3]*distance_scale*1e3; mean[4]*time_scale; mean[5:8]] # in meters

    satposes = vcat(sat1poses, sat2poses, sat3poses, sat4poses, sat5poses, sat6poses, sat7poses, sat8poses)
    
    return mean, all_r0, iters, satposes
end

receiver_pose (generic function with 1 method)

In [53]:
size(eci_1)[2]

5670

In [54]:
#Navigation
#count = 0
sat_allpose = zeros(32,1000)

all_time = zeros(4, size(eci_1)[2])

n=1000

PDOP_array = zeros(size(eci_1)[2],1)

#for i=1:size(eci_1)[2]

for i=215
    
    TEC = rand(d,4) #vTEC for each satellite from custom distribution
    
    #TEC = [5e-6, 2.4e-6, 3.7e-6, 4.1e-6]
    
    r0_TEC= vec([r0_scaled TEC])
    
    satposes = [eci_1[1,i] eci_1[2,i] eci_1[3,i];eci_2[1,i] eci_2[2,i] eci_2[3,i];eci_3[1,i] eci_3[2,i] eci_3[3,i];eci_4[1,i] eci_4[2,i] eci_4[3,i]]

    elev_angles = elevation_angle(satposes, r0[1:3])
    
    println("elevation angles: ", elev_angles)

    #check all angles to see if it is on the horizon 
    
    inhorizon = all(x->x<70, elev_angles)
    
    if inhorizon==false
        #PDOP_array[i,1] = NaN
        continue #skip the current iteration
    end
#i=1
    #count = count+1

    #In km then to custom units
    r1 = [eci_1[1,i], eci_1[2,i], eci_1[3,i], 0]*1e-3/distance_scale
    r2 = [eci_2[1,i], eci_2[2,i], eci_2[3,i], 0]*1e-3/distance_scale
    r3 = [eci_3[1,i], eci_3[2,i], eci_3[3,i], 0]*1e-3/distance_scale
    r4 = [eci_4[1,i], eci_4[2,i], eci_4[3,i], 0]*1e-3/distance_scale
    
    #measurments at 2nd frequency
    r5 = [eci_1[1,i], eci_1[2,i], eci_1[3,i], 0]*1e-3/distance_scale
    r6 = [eci_2[1,i], eci_2[2,i], eci_2[3,i], 0]*1e-3/distance_scale
    r7 = [eci_3[1,i], eci_3[2,i], eci_3[3,i], 0]*1e-3/distance_scale
    r8 = [eci_4[1,i], eci_4[2,i], eci_4[3,i], 0]*1e-3/distance_scale

    #t = measurment(r0_TEC, r1,r2,r3,r4, elev_angles)
    
    satposes = vcat(r1',r2',r3',r4')
    
    t = get_time(r0_TEC, satposes, elev_angles) #get the time at the actual position
    
    all_time[:, i] = t[1:4]
    
    #set the time measurments
    
    r1[4] = t[1] 
    r2[4] = t[2]
    r3[4] = t[3]
    r4[4] = t[4]
    r5[4] = t[5] 
    r6[4] = t[6]
    r7[4] = t[7]
    r8[4] = t[8]
    
    all_sats_scaled = vcat(r1',r2',r3',r4', r5', r6', r7', r8')
    
    #Generate a guess at the centroid of all 4 sats on the surface of the Earth. Scaled to km
    
    centroid_guess = [(eci_1[1,i]+eci_2[1,i]+eci_3[1,i]+eci_4[1,i])/4, (eci_1[2,i]+eci_2[2,i]+eci_3[2,i]+eci_4[2,i])/4, (eci_1[3,i]+eci_2[3,i]+eci_3[3,i]+eci_4[3,i])/4] 
    #centroid_guess = eci_2[1:3, i]
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    xyz_guess = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3 #change to km

    #Find all possible positions from montecarlo and send unscaled guess
    mean, all_r0, iters, sat_allposes = receiver_pose(all_sats_scaled, xyz_guess, elev_angles, TEC)
    
    sat_allpose = sat_allposes
    
    println(size(all_r0))
    
    #all_r0_scaled = zeros(n,8)

    #Rescale back to units (m and s)
    #for i in 1:n
    
    #    all_r0_scaled[i,:] = [all_r0[i][1:3]*distance_scale*1e3; all_r0[i][4]*time_scale; all_r0[i][5:8]]
        
    #end
    
    #println(sizeall_r0[1])
    
    #println(size(all_r0))
    
    totalsum = zeros(8,8)

    for i in 1:n
    
        value = all_r0[i] - mean
        
        #println("this is value: ", value)
        matrixvalue = value*transpose(value)
        totalsum += matrixvalue
    
    end

    covariancematrix = totalsum/n
    
    PDOP = sqrt(covariancematrix[1,1] + covariancematrix[2,2] + covariancematrix[3,3])
    TDOP = sqrt(covariancematrix[4,4])
    TEC_cov = sqrt(covariancematrix[5,5] + covariancematrix[6,6] + covariancematrix[7,7] + covariancematrix[8,8])
    PDOP_array[i,1] = PDOP
    
    println("This is covariance diagonal: ", diag(covariancematrix))
    println("cov position erors scaled: ", PDOP)
    println("cov position erors in meters: ", PDOP*1e3*distance_scale)
    println("cov time erors: ", TDOP)
    println("cov TEC erors: ", TEC_cov)
    println("Mean (m): ", mean)
    println("Iterations: ", iters)
    println("Count: ", i)

end    

elevation angles: [53.283388114452364, 50.40844562600919, 31.213336458891163, 65.2162266423407]
(1000,)
This is covariance diagonal: [9.024316781994319e-11, 1.1387215743107396e-10, 2.2205184024784215e-12, 2.3014179187276413e-11, 1.504517461602969e-11, 1.612076017572134e-11, 2.634298320977172e-11, 9.199373369780594e-12]
cov position erors scaled: 1.4364394997823456e-5
cov position erors in meters: 91.6180691631562
cov time erors: 4.797309578011035e-6
cov TEC erors: 8.167514393700408e-6
Mean (m): [-0.9679477498853963, -0.25114924001274325, -1.027225960576264e-7, 0.00046966068131636834, 6.8645525909812644e-6, 5.289138151895358e-6, 9.857947764872584e-6, 1.0404971722333516e-5]
Iterations: 5
Count: 215


UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

...

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

In [29]:
sat_allpose

32×1000 Matrix{Float64}:
 -1.0384     -1.0384     -1.0384     …  -1.0384     -1.0384     -1.0384
 -0.271343   -0.271343   -0.271343      -0.271343   -0.271343   -0.271343
 -0.0982209  -0.0982209  -0.0982209     -0.0982209  -0.0982209  -0.0982209
  0.123023    0.123023    0.123024       0.123025    0.123024    0.123023
 -1.03963    -1.03963    -1.03963       -1.03963    -1.03963    -1.03963
 -0.269499   -0.269499   -0.269499   …  -0.269499   -0.269499   -0.269499
  0.0894671   0.0894672   0.0894671      0.0894671   0.0894671   0.0894671
  0.116574    0.116575    0.116576       0.116574    0.116575    0.116576
 -1.0372     -1.0372     -1.0372        -1.0372     -1.0372     -1.0372
 -0.289803   -0.289803   -0.289803      -0.289803   -0.289803   -0.289803
 -0.0419685  -0.0419685  -0.0419685  …  -0.0419685  -0.0419685  -0.0419685
  0.0902031   0.0902016   0.0902016      0.0902014   0.0902025   0.090202
 -1.029      -1.029      -1.029         -1.029      -1.029      -1.029
  ⋮               

In [30]:
println(size(sat_allpose))
using LinearAlgebra

(32, 1000)


In [31]:
#Get Pin 
#Linear Covariance Calculation
n = 1000
totalsum1 = zeros(4,4)
totalsum2 = zeros(4,4)
totalsum3 = zeros(4,4)
totalsum4 = zeros(4,4)

totalsum5 = zeros(4,4)
totalsum6 = zeros(4,4)
totalsum7 = zeros(4,4)
totalsum8 = zeros(4,4)

#satellite positions in meters and seconds
sat1poses_t = zeros(4,1000)
sat2poses_t = zeros(4,1000)
sat3poses_t = zeros(4,1000)
sat4poses_t = zeros(4,1000)

sat5poses_t = zeros(4,1000)
sat6poses_t = zeros(4,1000)
sat7poses_t = zeros(4,1000)
sat8poses_t = zeros(4,1000)


#Rescale data back to meters (m)
sat1poses_t[1:3, :] = sat_allpose[1:3, :]
sat2poses_t[1:3, :] = sat_allpose[5:7, :]
sat3poses_t[1:3, :] = sat_allpose[9:11, :]
sat4poses_t[1:3, :] = sat_allpose[13:15, :]

sat5poses_t[1:3, :] = sat_allpose[17:19, :]
sat6poses_t[1:3, :] = sat_allpose[21:23, :]
sat7poses_t[1:3, :] = sat_allpose[25:27, :]
sat8poses_t[1:3, :] = sat_allpose[29:31, :]

sat1poses_t[4, :] = sat_allpose[4,:]
sat2poses_t[4, :] = sat_allpose[8,:]
sat3poses_t[4, :] = sat_allpose[12,:]
sat4poses_t[4, :] = sat_allpose[16,:]

sat5poses_t[4, :] = sat_allpose[20,:]
sat6poses_t[4, :] = sat_allpose[24,:]
sat7poses_t[4, :] = sat_allpose[28,:]
sat8poses_t[4, :] = sat_allpose[32,:]




# println(sat1poses_t[:,1])
# println(sat1poses_t[:,2])
# println(sat1poses_t[:,3])

#println(n)
sat1mean = (sum(sat1poses_t, dims = 2)/n)
sat2mean = (sum(sat2poses_t, dims = 2)/n)
sat3mean = (sum(sat3poses_t, dims = 2)/n)
sat4mean = (sum(sat4poses_t, dims = 2)/n)
sat5mean = (sum(sat5poses_t, dims = 2)/n)
sat6mean = (sum(sat6poses_t, dims = 2)/n)
sat7mean = (sum(sat7poses_t, dims = 2)/n)
sat8mean = (sum(sat8poses_t, dims = 2)/n)


println(sat1mean)
# println(sat2mean)
# println(sat3mean)
# println(sat4mean)
# println(sat5mean)
# println(sat6mean)
# println(sat7mean)

# sat5mean = sum(sat1poses, dims = 2)/n
# sat6mean = sum(sat2poses, dims = 2)/n
# sat7mean = sum(sat3poses, dims = 2)/n
# sat8mean = sum(sat4poses, dims = 2)/n

for i in 1:n
    
    value1 = sat1poses_t[:,i] - sat1mean
    matrixvalue1 = value1*transpose(value1)
    totalsum1 += matrixvalue1
    
    value2 = sat2poses_t[:,i] - sat2mean
    matrixvalue2 = value2*transpose(value2)
    totalsum2 += matrixvalue2
    
    value3 = sat3poses_t[:,i] - sat3mean
    matrixvalue3 = value3*transpose(value3)
    totalsum3 += matrixvalue3
    
    value4 = sat4poses_t[:,i] - sat4mean
    matrixvalue4 = value4*transpose(value4)
    totalsum4 += matrixvalue4
    
    value5 = sat5poses_t[:,i] - sat5mean
    matrixvalue5 = value5*transpose(value5)
    totalsum5 += matrixvalue5
    
    value6 = sat6poses_t[:,i] - sat6mean
    matrixvalue6 = value6*transpose(value6)
    totalsum6 += matrixvalue6
    
    value7 = sat7poses_t[:,i] - sat7mean
    matrixvalue7 = value7*transpose(value7)
    totalsum7 += matrixvalue7
    
    value8 = sat8poses_t[:,i] - sat8mean
    matrixvalue8 = value8*transpose(value8)
    totalsum8 += matrixvalue8
end

covariance_sat1 = totalsum1/n
covariance_sat2 = totalsum2/n
covariance_sat3 = totalsum3/n
covariance_sat4 = totalsum4/n

covariance_sat5 = totalsum5/n
covariance_sat6 = totalsum6/n
covariance_sat7 = totalsum7/n
covariance_sat8 = totalsum8/n

println("covariance of sat 1: ", diag(covariance_sat1))

#println("covariance of sat 4: ", diag(covariance_sat4))

Pin = BlockDiagonal([covariance_sat1, covariance_sat2, covariance_sat3, covariance_sat4, covariance_sat5, covariance_sat6, covariance_sat7, covariance_sat8])

[-1.0384039405326535; -0.2713432312547311; -0.09822090877570758; 0.12302459040289693]
covariance of sat 1: [2.1778439164746363e-16, 2.3492347487495e-16, 2.457893882102578e-16, 8.789695935106998e-13]


32×32 BlockDiagonal{Float64, Matrix{Float64}}:
  2.17784e-16  2.90331e-18  3.4048e-19   …   0.0           0.0
  2.90331e-18  2.34923e-16  2.99046e-18      0.0           0.0
  3.4048e-19   2.99046e-18  2.45789e-16      0.0           0.0
 -4.67643e-16  6.27928e-17  6.70162e-17      0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  0.0          0.0          0.0          …   0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  0.0          0.0          0.0          …   0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  0.0          0.0          0.0              0.0           0.0
  ⋮                                      ⋱   ⋮            
  0.0          0.0          0.0          …   0.0           0.0
  0.0       

In [61]:
#write to a file. Same covariance matrix for each time step. 
using DelimitedFiles

open("sat_cov_TOA_2freq.txt", "w") do file
        writedlm(file, Pin)
    end

In [32]:
#reading 

Pin_read = readdlm("sat_cov_TOA_2freq.txt")

32×32 Matrix{Float64}:
  2.52647e-16   1.33949e-18  -1.68075e-18  …   0.0           0.0
  1.33949e-18   2.54946e-16  -1.22305e-17      0.0           0.0
 -1.68075e-18  -1.22305e-17   2.48618e-16      0.0           0.0
 -6.68307e-17  -1.75375e-16  -5.04493e-16      0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              0.0           0.0
  ⋮                                        ⋱   ⋮            
  0.0           0.0           0.0          …   0.0           0.0
  0.0 

In [63]:
#TEC = rand(d,4)

In [33]:
#Pose at Time 151
count = 215

#TEC = [5e-6, 2.4e-6, 3.7e-6, 4.1e-6]

TEC = rand(d,4) #vTEC for each satellite from custom distribution

#TEC = [4e-6, 4.2e-6, 5e-6, 3e-6]

#println("this is TEC: ", TEC)
r0_TEC= vec([r0_scaled TEC]) #known position

#println("this is r0_TEC: ", r0_TEC)

satposes_m = [eci_1[1,count] eci_1[2,count] eci_1[3,count];eci_2[1,count] eci_2[2,count] eci_2[3,count];eci_3[1,count] eci_3[2,count] eci_3[3,count];eci_4[1,count] eci_4[2,count] eci_4[3,count]]

elev_angles = elevation_angle(satposes_m, r0[1:3])

#In km then to custom units
r1 = [eci_1[1,count], eci_1[2,count], eci_1[3,count], 0]*1e-3/distance_scale
r2 = [eci_2[1,count], eci_2[2,count], eci_2[3,count], 0]*1e-3/distance_scale
r3 = [eci_3[1,count], eci_3[2,count], eci_3[3,count], 0]*1e-3/distance_scale
r4 = [eci_4[1,count], eci_4[2,count], eci_4[3,count], 0]*1e-3/distance_scale
    
# #measurments at 2nd frequency
r5 = [eci_1[1,count], eci_1[2,count], eci_1[3,count], 0]*1e-3/distance_scale
r6 = [eci_2[1,count], eci_2[2,count], eci_2[3,count], 0]*1e-3/distance_scale
r7 = [eci_3[1,count], eci_3[2,count], eci_3[3,count], 0]*1e-3/distance_scale
r8 = [eci_4[1,count], eci_4[2,count], eci_4[3,count], 0]*1e-3/distance_scale

#t = measurment(r0_TEC, r1,r2,r3,r4, elev_angles)
    
satposes = vcat(r1',r2',r3',r4')
    
t = get_time(r0_TEC, satposes, elev_angles)
  
#set the time measurments
    
r1[4] = t[1] 
r2[4] = t[2]
r3[4] = t[3]
r4[4] = t[4]
r5[4] = t[5] 
r6[4] = t[6]
r7[4] = t[7]
r8[4] = t[8]

sat_pose_t = vcat(r1',r2',r3',r4', r5', r6', r7', r8')

#get sat_pose_t in meters and seconds to find the covariance later
#sat_pose_ms = zeros(8,4)

#sat_pose_ms[1,:] = [sat_pose_t[1, 1:3]*1e3*distance_scale; sat_pose_t[1,4]*time_scale]
#sat_pose_ms[2,:] = [sat_pose_t[2, 1:3]*1e3*distance_scale; sat_pose_t[2,4]*time_scale]
#sat_pose_ms[3,:] = [sat_pose_t[3, 1:3]*1e3*distance_scale; sat_pose_t[3,4]*time_scale]
#sat_pose_ms[4,:] = [sat_pose_t[4, 1:3]*1e3*distance_scale; sat_pose_t[4,4]*time_scale]
#sat_pose_ms[5,:] = [sat_pose_t[5, 1:3]*1e3*distance_scale; sat_pose_t[5,4]*time_scale]
#sat_pose_ms[6,:] = [sat_pose_t[6, 1:3]*1e3*distance_scale; sat_pose_t[6,4]*time_scale]
#sat_pose_ms[7,:] = [sat_pose_t[7, 1:3]*1e3*distance_scale; sat_pose_t[7,4]*time_scale]
#sat_pose_ms[8,:] = [sat_pose_t[8, 1:3]*1e3*distance_scale; sat_pose_t[8,4]*time_scale]


centroid_guess = [(eci_1[1,count]+eci_2[1,count]+eci_3[1,count]+eci_4[1,count])/4, (eci_1[2,count]+eci_2[2,count]+eci_3[2,count]+eci_4[2,count])/4, (eci_1[3,count]+eci_2[3,count]+eci_3[3,count]+eci_4[3,count])/4] 
#centroid_guess = eci_2[1:3, i]
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

guess = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3 #change to km

#initial_x0 = [r0_TEC[1]+100,r0_TEC[2]+100,r0_TEC[3]+100,r0_TEC[4]+100,r0_TEC[5]+100,r0_TEC[6]+100,r0_TEC[7]+100,r0_TEC[8]+100 ]
initial_x0 = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 0.0001/time_scale, 3e-6, 3.5e-6, 2e-6, 4e-6]

8-element Vector{Float64}:
 -0.9653668525091973
 -0.25996077873057377
  0.022078340022466097
  0.004700314384940316
  3.0e-6
  3.5e-6
  2.0e-6
  4.0e-6

In [65]:
# jacobian2 = ForwardDiff.jacobian(dall_sats_scaled -> residual(initial_x0, dall_sats_scaled, elev_angles), sat_pose_t)

# println(jacobian2[1,:])

In [ ]:
function usingforwarddiffjacobian(all_sats_scaled, x0)
    
    b = 0.01
    c_ =0.5
    β = 10
    α = 1
    
    
    X = NaN*[zeros(8) for i = 1:1000]
    R = NaN*[zeros(8) for i = 1:1000]
    
    X[1] = x0

    iters = 0
    
    for k=1:1000

        R[k] = residual(X[k], all_sats_scaled, elev_angles)


        iters += 1

        #original tolerancec 1e-12

        if(norm(R[k]) < 1e-10)

            break

        end

        jacobian = ForwardDiff.jacobian(dx -> residual(dx, all_sats_scaled, elev_angles), X[k])

        conditionnum = cond(jacobian)

        deltax = (jacobian)\-R[k]

        while norm(residual(X[k] + α*deltax, all_sats_scaled, elev_angles)) > norm(residual(X[k], all_sats_scaled, elev_angles) + b*α*jacobian'*deltax)

            α = c_*α
        end
        X[k+1] = X[k] + α*deltax

    end 
    
    R = R[1:iters]
    X = X[1:iters]
    x_solution = X[end]
    x_solution_scaled = [x_solution[1]*1e3*distance_scale, x_solution[2]*1e3*distance_scale, x_solution[3]*1e3*distance_scale, x_solution[4]*time_scale,
                         x_solution[5], x_solution[6], x_solution[7], x_solution[8]]
   
    return x_solution_scaled

end

In [ ]:
x_solution = usingforwarddiffjacobian(sat_pose_t, initial_x0)

In [ ]:
using BenchmarkTools

In [ ]:
#@btime usingforwarddiffjacobian($sat_pose_t, $initial_x0) #run to test speed. 

In [34]:
#Least Squares Solution

#y is sat position
#x is the tag

function LS_Amatrix(x, y, scale, A, angle)
    #works
    #jacobian_pose = (x[1:3] - A'*y[1:3])/norm(y[1:3]-A*x[1:3])
    
    jacobian_pose = (x[1:3] - A*y[1:3])/norm(x[1:3]-A*y[1:3])
    
    #jacobian_pose = (x0[1:3] - r_sat[1:3])/norm(r_sat[1:3]-x0[1:3])
    
    dpdtau = c
    dpdTEC = (scale/cosd(angle))*(1e-3)/distance_scale
    
    return [jacobian_pose; dpdtau; dpdTEC]
end

LS_Amatrix (generic function with 1 method)

In [35]:
function LS_Amatrix_dpdy(x, y, scale, A, angle)
    
    #c = C_LIGHT
    
    #works
    jacobian_pose = (y[1:3] - A'*x[1:3])/norm(x[1:3]-A*y[1:3])
    
    dpdtau = -c
    
    return [jacobian_pose; dpdtau]
end

LS_Amatrix_dpdy (generic function with 1 method)

In [36]:
sat_pose_t

8×4 Matrix{Float64}:
 -1.0384   -0.271343  -0.0982209  0.123028
 -1.03963  -0.269499   0.0894671  0.116577
 -1.0372   -0.289803  -0.0419685  0.0902022
 -1.029    -0.285343   0.145503   0.161927
 -1.0384   -0.271343  -0.0982209  0.123025
 -1.03963  -0.269499   0.0894671  0.116573
 -1.0372   -0.289803  -0.0419685  0.0902005
 -1.029    -0.285343   0.145503   0.161925

In [37]:
function customjacobian(initial_x0,sat_pose_t, elev_angles)
    
    #println("this is initial_x0: ", initial_x0)
    #println("this is sat_pose_t: ", sat_pose_t)
    #println("this is elev angles: ", elev_angles)
    
    while norm(residual(initial_x0, sat_pose_t, elev_angles))>1e-10

        omega = OMEGA_EARTH*time_scale


        angles = zeros(4)

        for i=1:4
            angles[i] = asind((R_EARTH*sind(elev_angles[i]))/(R_EARTH + hi))
        end

        scale_f1 = (40.3/(f1)^2)*1e22
        scale_f2 = (40.3/(f2)^2)*1e22

        A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
              -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
              0 0 1];
        A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
              -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
              0 0 1];
        A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
              -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
              0 0 1];
        A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
              -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
              0 0 1];
        A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
              -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
              0 0 1];
        A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
              -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
              0 0 1];
        A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
              -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
              0 0 1];
        A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
              -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
              0 0 1];

        P_computed1  = norm(initial_x0[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - initial_x0[4]) + (scale_f1*(initial_x0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
        P_computed2  = norm(initial_x0[1:3] - A2*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - initial_x0[4]) + (scale_f1*(initial_x0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
        P_computed3  = norm(initial_x0[1:3] - A3*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - initial_x0[4]) + (scale_f1*(initial_x0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
        P_computed4  = norm(initial_x0[1:3] - A4*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - initial_x0[4]) + (scale_f1*(initial_x0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
        P_computed5  = norm(initial_x0[1:3] - A5*sat_pose_t[5,1:3]) - c*(sat_pose_t[5,4] - initial_x0[4]) + (scale_f2*(initial_x0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
        P_computed6  = norm(initial_x0[1:3] - A6*sat_pose_t[6,1:3]) - c*(sat_pose_t[6,4] - initial_x0[4]) + (scale_f2*(initial_x0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
        P_computed7  = norm(initial_x0[1:3] - A7*sat_pose_t[7,1:3]) - c*(sat_pose_t[7,4] - initial_x0[4]) + (scale_f2*(initial_x0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
        P_computed8  = norm(initial_x0[1:3] - A8*sat_pose_t[8,1:3]) - c*(sat_pose_t[8,4] - initial_x0[4]) + (scale_f2*(initial_x0[8]/cosd(angles[4])))*(1e-3)/distance_scale;

        #P_computed1 = norm(all_sats_scaled[1,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[1,4]
        #P_computed2 = norm(all_sats_scaled[2,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[2,4]
        #P_computed3 = norm(all_sats_scaled[3,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[3,4]
        #P_computed4 = norm(all_sats_scaled[4,1:3]-x0[1:3]) + c*x0[4] - c*all_sats_scaled[4,4]

        P_computed = [P_computed1, P_computed2, P_computed3, P_computed4, P_computed5, P_computed6, P_computed7, P_computed8]

        P_observed = zeros(8)

        deltaP = P_observed - P_computed
        
        println("this is deltaP: ", norm(deltaP))

        A = zeros(8,8)

        A_row1 = LS_Amatrix(initial_x0, sat_pose_t[1,:], scale_f1, A1, angles[1])'
        A_row2 = LS_Amatrix(initial_x0, sat_pose_t[2,:], scale_f1, A2, angles[2])'
        A_row3 = LS_Amatrix(initial_x0, sat_pose_t[3,:], scale_f1, A3, angles[3])'
        A_row4 = LS_Amatrix(initial_x0, sat_pose_t[4,:], scale_f1, A4, angles[4])'
        A_row5 = LS_Amatrix(initial_x0, sat_pose_t[5,:], scale_f2, A5, angles[1])'
        A_row6 = LS_Amatrix(initial_x0, sat_pose_t[6,:], scale_f2, A6, angles[2])'
        A_row7 = LS_Amatrix(initial_x0, sat_pose_t[7,:], scale_f2, A7, angles[3])'
        A_row8 = LS_Amatrix(initial_x0, sat_pose_t[8,:], scale_f2, A8, angles[4])'

        #println("this is A_row1: ", A_row1)

        A[1,:] = [A_row1[1:5]; zeros(3)]
        A[2,:] = [A_row2[1:4];0.0; A_row2[5]; zeros(2)]
        A[3,:] = [A_row3[1:4];zeros(2); A_row3[5]; 0.0]
        A[4,:] = [A_row4[1:4];zeros(3); A_row4[5]]
        A[5,:] = [A_row5[1:5]; zeros(3)]
        A[6,:] = [A_row6[1:4];0.0; A_row6[5]; zeros(2)]
        A[7,:] = [A_row7[1:4];zeros(2); A_row7[5]; 0.0]
        A[8,:] = [A_row8[1:4];zeros(3); A_row8[5]]

        #println("this is the jacobian: ", A)

        #println(size(A))
    #     A = [A_row1 zeros(3)';
    #          A_row2[1:4] A_row2[5] zeros(3)']

        #; A_row2[1:4] 0 A_row2[5] zeros(3)'

    #     println(A_row1)
    #     println(A_row2)
    #     println(A)
    #     println(size(A))
    #     println(A_row2)
    #     println(A_row3)
    #     println(A_row4)

        #Need to reformat this A matrix
        #A = vcat(LS_Amatrix(x0, sat_pose_t[1,:], scale_f1, A1, angles[1])', LS_Amatrix(x0, sat_pose_t[2,:], scale_f1, angles[2])', LS_Amatrix(x0, sat_pose_t[3,:], scale_f1, angles[3])', LS_Amatrix(x0, sat_pose_t[4,:], scale_f1, angles[4])', 
        #         LS_Amatrix(x0, sat_pose_t[1,:], scale_f2, angles[1])', LS_Amatrix(x0, sat_pose_t[2,:], scale_f2, angles[2])', LS_Amatrix(x0, sat_pose_t[3,:], scale_f2, angles[3])', LS_Amatrix(x0, sat_pose_t[4,:], scale_f2, angles[4])')

        F = qr(A)

        d = transpose(F.Q)*deltaP

        deltax = F.R\d

        #theresidual = residual(initial_x0, sat_pose_t, elev_angles)

        #println(theresidual)

        #println("this is the residual: ", norm(theresidual))


        #println("this is delta x: ", deltax)
        initial_x0 = initial_x0+deltax


    end

    x_sol = initial_x0
    #x_sol_rescaled = [x_sol[1]*distance_scale*1e3,x_sol[2]*distance_scale*1e3,x_sol[3]*distance_scale*1e3, x_sol[4]*time_scale, x_sol[5], x_sol[6], x_sol[7], x_sol[8]]
    
    return x_sol
    
end
#println("this is x_solution rescaled: ", x_sol_rescaled)

customjacobian (generic function with 1 method)

In [38]:
x_sol = customjacobian(initial_x0,sat_pose_t, elev_angles)

this is deltaP: 0.04770670892115325
this is deltaP: 0.004905702671613171
this is deltaP: 0.00018467211580203185
this is deltaP: 2.536574853852513e-7


8-element Vector{Float64}:
 -0.9679486083850798
 -0.25114878264887336
  2.745689843842618e-15
  0.00047003143868175605
  8.348409621847621e-6
  1.1321494802394232e-5
  6.723004536258151e-6
  4.128205978903478e-6

In [ ]:
#@btime customjacobian($initial_x0,$sat_pose_t, $elev_angles)

In [ ]:
sat_pose_ms

In [ ]:
# satelliteposition = [-6.56223e6, -1.7204e6, -1.10968e6]
# tagposition = [-6.1737081556761805e6, -1.601861167311977e6, 2.3478067043705542e-8]

In [ ]:
println(OMEGA_EARTH)

In [39]:
omega = OMEGA_EARTH*time_scale #scaled

angles = zeros(4)

for i=1:4
    angles[i] = asind((R_EARTH*sind(elev_angles[i]))/(R_EARTH + hi))
end

scale_f1 = (40.3/(f1)^2)*1e22
scale_f2 = (40.3/(f2)^2)*1e22

A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
      -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
      0 0 1];
A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
      -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
      0 0 1];
A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
      -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
      0 0 1];
A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
      -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
      0 0 1];
A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
      -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
      0 0 1];
A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
      -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
      0 0 1];
A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
      -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
      0 0 1];
A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
      -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
      0 0 1];

#create dfdy
dfdx = zeros(8,8)

dfdx_row1 = LS_Amatrix(x_sol, sat_pose_t[1,:], scale_f1, A1, angles[1])'
dfdx_row2 = LS_Amatrix(x_sol, sat_pose_t[2,:], scale_f1, A2, angles[2])'
dfdx_row3 = LS_Amatrix(x_sol, sat_pose_t[3,:], scale_f1, A3, angles[3])'
dfdx_row4 = LS_Amatrix(x_sol, sat_pose_t[4,:], scale_f1, A4, angles[4])'
dfdx_row5 = LS_Amatrix(x_sol, sat_pose_t[5,:], scale_f2, A5, angles[1])'
dfdx_row6 = LS_Amatrix(x_sol, sat_pose_t[6,:], scale_f2, A6, angles[2])'
dfdx_row7 = LS_Amatrix(x_sol, sat_pose_t[7,:], scale_f2, A7, angles[3])'
dfdx_row8 = LS_Amatrix(x_sol, sat_pose_t[8,:], scale_f2, A8, angles[4])'

#TEC term must be scaled 
#dfdx_row1[5] = dfdx_row1[5]*1e3*distance_scale
#dfdx_row2[5] = dfdx_row2[5]*1e3*distance_scale
#dfdx_row3[5] = dfdx_row3[5]*1e3*distance_scale
#dfdx_row4[5] = dfdx_row4[5]*1e3*distance_scale
#dfdx_row5[5] = dfdx_row5[5]*1e3*distance_scale
#dfdx_row6[5] = dfdx_row6[5]*1e3*distance_scale
#dfdx_row7[5] = dfdx_row7[5]*1e3*distance_scale
#dfdx_row8[5] = dfdx_row8[5]*1e3*distance_scale

#C must be scaled
# dfdx_row1[4] = dfdx_row1[5]*C_LIGHT #Needs to be the actual speed of light in m/s instead of 1
# dfdx_row2[4] = dfdx_row2[5]*C_LIGHT
# dfdx_row3[4] = dfdx_row3[5]*C_LIGHT
# dfdx_row4[4] = dfdx_row4[5]*C_LIGHT
# dfdx_row5[4] = dfdx_row5[5]*C_LIGHT
# dfdx_row6[4] = dfdx_row6[5]*C_LIGHT
# dfdx_row7[4] = dfdx_row7[5]*C_LIGHT
# dfdx_row8[4] = dfdx_row8[5]*C_LIGHT

dfdx[1,:] = [dfdx_row1[1:5]; zeros(3)]
dfdx[2,:] = [dfdx_row2[1:4];0.0; dfdx_row2[5]; zeros(2)]
dfdx[3,:] = [dfdx_row3[1:4];zeros(2); dfdx_row3[5]; 0.0]
dfdx[4,:] = [dfdx_row4[1:4];zeros(3); dfdx_row4[5]]
dfdx[5,:] = [dfdx_row5[1:5]; zeros(3)]
dfdx[6,:] = [dfdx_row6[1:4];0.0; dfdx_row6[5]; zeros(2)]
dfdx[7,:] = [dfdx_row7[1:4];zeros(2); dfdx_row7[5]; 0.0]
dfdx[8,:] = [dfdx_row8[1:4];zeros(3); dfdx_row8[5]]

#derivative wrt measurements (satellite position and times)
dfdy = zeros(8,32)

#create dfdx
dfdy[1, 1:4] = LS_Amatrix_dpdy(x_sol, sat_pose_t[1,:], scale_f1, A1, angles[1])'[1:4]
dfdy[2, 5:8] = LS_Amatrix_dpdy(x_sol, sat_pose_t[2,:], scale_f1, A2, angles[2])'[1:4]
dfdy[3, 9:12] = LS_Amatrix_dpdy(x_sol, sat_pose_t[3,:], scale_f1, A3, angles[3])'[1:4]
dfdy[4, 13:16] = LS_Amatrix_dpdy(x_sol, sat_pose_t[4,:], scale_f1, A4, angles[4])'[1:4]
dfdy[5, 17:20] = LS_Amatrix_dpdy(x_sol, sat_pose_t[5,:], scale_f2, A5, angles[1])'[1:4]
dfdy[6, 21:24] = LS_Amatrix_dpdy(x_sol, sat_pose_t[6,:], scale_f2, A6, angles[2])'[1:4]
dfdy[7, 25:28] = LS_Amatrix_dpdy(x_sol, sat_pose_t[7,:], scale_f2, A7, angles[3])'[1:4]
dfdy[8, 29:32] = LS_Amatrix_dpdy(x_sol, sat_pose_t[8,:], scale_f2, A8, angles[4])'[1:4]

A = -inv(dfdx)*dfdy
#println("this is dfdx: ", dfdx)
#dfdy = vcat(LS_Amatrix(x_sol, all_sats_scaled[1,:])',LS_Amatrix(x_sol, all_sats_scaled[2,:])',LS_Amatrix(x_sol, all_sats_scaled[3,:])',LS_Amatrix(x_sol, all_sats_scaled[4,:])')

8×32 Matrix{Float64}:
  0.397802      0.11402       0.55457      …   5.77372      -6.40668
  1.07523       0.308188      1.49897         -4.05601       4.50066
 -0.286405     -0.0820907    -0.399273         0.17347      -0.192487
 -0.636251     -0.182365     -0.886989        -2.78999       3.09585
  1.70338       0.488231      2.37466         -6.15142e-10   6.82578e-10
  1.53095e-10   4.38809e-11   2.13428e-10  …  -8.72469e-10   9.68115e-10
  9.97312e-11   2.85855e-11   1.39034e-10     -7.55874e-10   8.38739e-10
  3.66094e-11   1.04932e-11   5.10367e-11      2.09153      -2.32082

In [43]:
dfdx

8×8 Matrix{Float64}:
 0.5749    0.16478    0.80146   1.0  0.60751   0.0       0.0       0.0
 0.617405  0.158051  -0.770604  1.0  0.0       0.578282  0.0       0.0
 0.771791  0.430781   0.467725  1.0  0.0       0.0       0.453387  0.0
 0.378124  0.211791  -0.901203  1.0  0.0       0.0       0.0       0.775586
 0.5749    0.16478    0.80146   1.0  0.270004  0.0       0.0       0.0
 0.617405  0.158051  -0.770604  1.0  0.0       0.257014  0.0       0.0
 0.771791  0.430781   0.467725  1.0  0.0       0.0       0.201505  0.0
 0.378124  0.211791  -0.901203  1.0  0.0       0.0       0.0       0.344705

In [53]:
dfdy

println(dfdy[1,:])

[-0.574899568942969, -0.16478050930178295, -0.8014598364131751, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [40]:
C_LIGHT

2.99792458e8

In [54]:
sat_pose_t

8×4 Matrix{Float64}:
 -1.0384   -0.271343  -0.0982209  0.123028
 -1.03963  -0.269499   0.0894671  0.116577
 -1.0372   -0.289803  -0.0419685  0.0902022
 -1.029    -0.285343   0.145503   0.161927
 -1.0384   -0.271343  -0.0982209  0.123025
 -1.03963  -0.269499   0.0894671  0.116573
 -1.0372   -0.289803  -0.0419685  0.0902005
 -1.029    -0.285343   0.145503   0.161925

In [60]:
function residual_vector(x, sat_pose_t, zenith)
    
    omega = OMEGA_EARTH*time_scale
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    A1 = [cos(omega*sat_pose_t[4]) sin(omega*sat_pose_t[4]) 0;
          -sin(omega*sat_pose_t[4]) cos(omega*sat_pose_t[4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[8]) sin(omega*sat_pose_t[8]) 0;
          -sin(omega*sat_pose_t[8]) cos(omega*sat_pose_t[8]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[12]) sin(omega*sat_pose_t[12]) 0;
          -sin(omega*sat_pose_t[12]) cos(omega*sat_pose_t[12]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[16]) sin(omega*sat_pose_t[16]) 0;
          -sin(omega*sat_pose_t[16]) cos(omega*sat_pose_t[16]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[20]) sin(omega*sat_pose_t[20]) 0;
          -sin(omega*sat_pose_t[20]) cos(omega*sat_pose_t[20]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[24]) sin(omega*sat_pose_t[24]) 0;
          -sin(omega*sat_pose_t[24]) cos(omega*sat_pose_t[24]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[28]) sin(omega*sat_pose_t[28]) 0;
          -sin(omega*sat_pose_t[28]) cos(omega*sat_pose_t[28]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[32]) sin(omega*sat_pose_t[32]) 0;
          -sin(omega*sat_pose_t[32]) cos(omega*sat_pose_t[32]) 0;
          0 0 1];
    
    res1 = norm(x[1:3] - A1*sat_pose_t[1:3]) - c*(sat_pose_t[4] - x[4]) + (scale_f1*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(x[1:3] - A2*sat_pose_t[5:7]) - c*(sat_pose_t[8] - x[4]) + (scale_f1*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(x[1:3] - A3*sat_pose_t[9:11]) - c*(sat_pose_t[12] - x[4]) + (scale_f1*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(x[1:3] - A4*sat_pose_t[13:15]) - c*(sat_pose_t[16] - x[4]) + (scale_f1*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(x[1:3] - A5*sat_pose_t[17:19]) - c*(sat_pose_t[20] - x[4]) + (scale_f2*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(x[1:3] - A6*sat_pose_t[21:23]) - c*(sat_pose_t[24] - x[4]) + (scale_f2*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(x[1:3] - A7*sat_pose_t[25:27]) - c*(sat_pose_t[28] - x[4]) + (scale_f2*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(x[1:3] - A8*sat_pose_t[29:31]) - c*(sat_pose_t[32] - x[4]) + (scale_f2*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

residual_vector (generic function with 1 method)

In [61]:
satpose_vec = vec(sat_pose_t')

32-element reshape(adjoint(::Matrix{Float64}), 32) with eltype Float64:
 -1.038403941200568
 -0.2713432311445819
 -0.0982209084540021
  0.12302760549025431
 -1.039629209856715
 -0.2694987113252471
  0.08946711621764344
  0.11657650731428386
 -1.0372006775085478
 -0.28980253228901287
 -0.04196852473216833
  0.09020218801125379
 -1.0289979884937088
  ⋮
 -1.039629209856715
 -0.2694987113252471
  0.08946711621764344
  0.1165728700830448
 -1.0372006775085478
 -0.28980253228901287
 -0.04196852473216833
  0.09020049461115617
 -1.0289979884937088
 -0.2853434293240319
  0.14550261624143093
  0.16192517252600397

In [64]:
dfdy_fd = ForwardDiff.jacobian(dall_sats_scaled -> residual_vector(x_sol, dall_sats_scaled, elev_angles), satpose_vec)


#dfdx_fd = ForwardDiff.jacobian(dx -> residual(dx, sat_pose_t, elev_angles), x_sol)

# A_forwardDiff = -inv(dfdx_fd)*dfdy_fd

8×32 Matrix{Float64}:
 -0.5749  -0.164781  -0.80146  -1.0  -0.0       …  -0.0       -0.0       -0.0
  0.0      0.0        0.0       0.0  -0.617405      0.0        0.0        0.0
 -0.0     -0.0       -0.0      -0.0  -0.0          -0.0       -0.0       -0.0
  0.0      0.0        0.0       0.0   0.0           0.0        0.0        0.0
 -0.0     -0.0       -0.0      -0.0  -0.0          -0.0       -0.0       -0.0
  0.0      0.0        0.0       0.0   0.0       …   0.0        0.0        0.0
 -0.0     -0.0       -0.0      -0.0  -0.0          -0.0       -0.0       -0.0
  0.0      0.0        0.0       0.0   0.0          -0.211791   0.901203  -1.0

In [65]:
dfdy

8×32 Matrix{Float64}:
 -0.5749  -0.164781  -0.80146  -1.0   0.0       …   0.0       0.0        0.0
  0.0      0.0        0.0       0.0  -0.617405      0.0       0.0        0.0
  0.0      0.0        0.0       0.0   0.0           0.0       0.0        0.0
  0.0      0.0        0.0       0.0   0.0           0.0       0.0        0.0
  0.0      0.0        0.0       0.0   0.0           0.0       0.0        0.0
  0.0      0.0        0.0       0.0   0.0       …   0.0       0.0        0.0
  0.0      0.0        0.0       0.0   0.0           0.0       0.0        0.0
  0.0      0.0        0.0       0.0   0.0          -0.211791  0.901203  -1.0

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

UndefVarError: UndefVarError: setexcludinghandlers! not defined

In [42]:
#Covariance from LS method
covariance_tag_derived = A*Pin*transpose(A)

println(diag(covariance_tag_derived))
#covariance_tag_fd = A_forwardDiff*Pin*transpose(A_forwardDiff)

# println(diag(covariance_tag_derived))
# println(diag(covariance_tag_fd))

#PDOP_LS_fd = sqrt(covariance_tag_fd[1,1] + covariance_tag_fd[2,2] + covariance_tag_fd[3,3])

PDOP_LS = sqrt(covariance_tag_derived[1,1] + covariance_tag_derived[2,2] + covariance_tag_derived[3,3])

TDOP_LS = sqrt(covariance_tag_derived[4,4])


println("PDOP with scaling: ", PDOP_LS)
PDOP_LS_scaled = PDOP_LS*distance_scale*1e3
#println(PDOP_LS_fd)
println("PDOP in meters: ", PDOP_LS_scaled)
#println(TDOP_LS)

[9.53024293554548e-11, 1.1668249636594155e-10, 2.2791968317853125e-12, 2.444702622529612e-11, 1.5307246318054455e-11, 1.6406552630150386e-11, 2.862102508598803e-11, 9.765516059642049e-12]
PDOP with scaling: 1.4637763577581846e-5
PDOP in meters: 93.36165122499264


In [ ]:
cov151 = [1334.1551554696332, 9314.841749109753, 274.088637333827, 
          1.5273003817106035e-14, 7.1755589311462015e-12, 3.458805910575227e-11,
          1.192840393243596e-11, 2.138823189152909e-11]

In [ ]:
println(covariance_tag)

In [ ]:
final_PDOP = PDOP_array[PDOP_array .!=0]

PDOP_max = findmax(final_PDOP)[1]
PDOP_min = findmin(final_PDOP)[1]

println("Maximum PDOP: ", PDOP_max)
println("Minimum PDOP: ", PDOP_min)

In [ ]:
final_PDOP = PDOP_array[PDOP_array .!=0]

PDOP_max = findmax(final_PDOP)[1]
PDOP_min = findmin(final_PDOP)[1]

println("Maximum PDOP: ", PDOP_max)
println("Minimum PDOP: ", PDOP_min)

In [ ]:
#From previous test

#Maximum PDOP: 1.8460672866561032
#Minimum PDOP: 0.7628220671844341


#tag =[-6.128804e6, -1.590206e6, 776.1502e3] true tag position

In [ ]:
#write to a file
open("sat_data.txt", "w") do file
        writedlm(file, PDOP_array)
    end

In [ ]:
x = log(178.1)

In [ ]:
log(2)

In [ ]:
k = 3000

centroid_guess = [(eci_1[1,k]+eci_2[1,k]+eci_3[1,k]+eci_4[1,k])/4, (eci_1[2,k]+eci_2[2,k]+eci_3[2,k]+eci_4[2,k])/4, (eci_1[3,k]+eci_2[3,k]+eci_3[3,k]+eci_4[3,k])/4] 
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

#Guess
g_xyz = sGEOCtoECEF(geodetic, use_degrees = true)


#Plot Satellite 1 and 2 Orbit
sat1 = scatter(x=eci_1[1,:], y=eci_1[2,:], z=eci_1[3,:], type="scatter3d", mode="lines", name="orbit 1&2")

#Plot Satellite 3 and 4 Orbit
sat3 = scatter(x=eci_3[1,:], y=eci_3[2,:], z=eci_3[3,:], type="scatter3d", mode="lines", name="orbit 3&4")

#Plot all 4 satellite initial positions
satellites = scatter(x=[eci_1[1,k],eci_2[1,k], eci_3[1,k], eci_4[1,k]], y=[eci_1[2,k],eci_2[2,k], eci_3[2,k], eci_4[2,k]],z=[eci_1[3,k],eci_2[3,k], eci_3[3,k], eci_4[3,k]], mode="markers", marker_size = 4, type="scatter3d", name="satellites")

#Plot Tag position
tag = scatter(x = [r0[1], r0[1]], y = [r0[2], r0[2]], z = [r0[3], r0[3]], type="scatter3d", name="tag", mode="markers", marker_size=5)

#Plot the Guess
guess = scatter(x = [g_xyz[1], g_xyz[1]], y = [g_xyz[2], g_xyz[2]], z = [g_xyz[3], g_xyz[3]], type="scatter3d", name="guess", mode="markers", marker_size=5)

#Plotting the sphere (Earth)
n = 100
u = range(-π, π; length = n)
v = range(0, π; length = n)
x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)' 
z = ones(n) * cos.(v)' 

earth = surface(z=z*6371000, x=x*6371000, y=y*6371000, showscale = false)

plot([sat1,sat3, satellites, tag, earth, guess])

In [ ]:
r0[1:3] - g_xyz 

In [ ]:
r0[1:3] - xyz

In [ ]:
g_xyz

In [ ]:
xyz

In [ ]:
xyz  + [0, 0, 1.5e6]

In [ ]:
satposes = [eci_1[1,k] eci_1[2,k] eci_1[3,k];eci_2[1,k] eci_2[2,k] eci_2[3,k];eci_3[1,k] eci_3[2,k] eci_3[3,k];eci_4[1,k] eci_4[2,k] eci_4[3,k]]
elev_angles = elevation_angle(satposes, r0[1:3])

In [ ]:
#For 199 count
#78.77006915596219
#38.78765448588974
#66.9345497258459
#61.301342621968

In [ ]:
rand(d_t,4)

In [ ]:
using LinearAlgebra

vector1 = [1,2,3]
vector2 = [2,9,8]

normfunc = norm(vector1-vector2)

In [ ]:
algebra = sqrt((vector1-vector2)^2)

In [ ]:
test = 299792458*1e-9

In [ ]:
time_scale

In [ ]:
#3 seconds. Making speed of light 1, scales the distance by radius of earth
distance = (3/time_scale)*1

In [ ]:
scaled = distance*distance_scale*1e-3

In [ ]:
distance = (3*299792458)

In [ ]:
norm(scaled-distance)